# 天池o2o优惠券使用预测比赛解析（初级）

**赛题链接：**

[天池o2o优惠券使用预测](https://tianchi.aliyun.com/getStart/introduction.htm?spm=5176.100066.0.0.518433afBqXIKM&raceId=231593)

## 导入相关库

In [1]:
# import libraries necessary for this project
import os, sys, pickle
 
import numpy as np
import pandas as pd
 
from datetime import date
 
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler
 
# display for this notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 导入数据

In [2]:
dfoff = pd.read_csv('data/ccf_offline_stage1_train.csv')
dfon = pd.read_csv('data/ccf_online_stage1_train.csv')
dftest = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')
 
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


### 简单统计

简单统计一下用户使用优惠券的情况：

In [3]:
print('有优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'] != 'null') & (dfoff['Date'] != 'null')].shape[0])
print('有优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'] != 'null') & (dfoff['Date'] == 'null')].shape[0])
print('无优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'] == 'null') & (dfoff['Date'] != 'null')].shape[0])
print('无优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'] == 'null') & (dfoff['Date'] == 'null')].shape[0])

有优惠卷，购买商品：75382
有优惠卷，未购商品：977900
无优惠卷，购买商品：701602
无优惠卷，未购商品：0


可见，很多人（701602）购买商品却没有使用优惠券，也有很多人（977900）有优惠券但却没有使用，真正使用优惠券购买商品的人（75382）很少！所以，这个比赛的意义就是把优惠券送给真正可能会购买商品的人。

## 特征提取

In [ ]:
dfoff.head(5)

### 打折率 Discount_rate

In [ ]:
print('Discount_rate 类型：\n',dfoff['Discount_rate'].unique())

打折率分为 3 种情况：

- 'null' 表示没有打折

- [0,1] 表示折扣率

- x:y 表示满x减y

**处理方式：**

- 打折类型：getDiscountType()

- 折扣率：convertRate()

- 满多少：getDiscountMan()

- 减多少：getDiscountJian()

In [4]:
# Convert Discount_rate and Distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)
    
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0
    
def processData(df):
    
    # convert discount_rate
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
    
    print(df['discount_rate'].unique())
    
    return df

In [5]:
dfoff = processData(dfoff)
dftest = processData(dftest)

[ 1.          0.86666667  0.95        0.9         0.83333333  0.8         0.5
  0.85        0.75        0.66666667  0.93333333  0.7         0.6
  0.96666667  0.98        0.99        0.975       0.33333333  0.2         0.4       ]
[ 0.83333333  0.9         0.96666667  0.8         0.95        0.75        0.98
  0.5         0.86666667  0.6         0.66666667  0.7         0.85
  0.33333333  0.94        0.93333333  0.975       0.99      ]


In [6]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,20
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,1
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,1
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,1


### 距离 Distance

In [7]:
print('Distance 类型：',dfoff['Distance'].unique())

Distance 类型： ['0' '1' 'null' '2' '10' '4' '7' '9' '3' '5' '6' '8']


将距离 str 转为 int。

In [8]:
# convert distance
dfoff['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dfoff['distance'].unique())
dftest['distance'] = dftest['Distance'].replace('null', -1).astype(int)
print(dftest['distance'].unique())

[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[ 1 -1  5  2  0 10  3  6  7  4  9  8]


In [9]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian,distance
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,20,1
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,1,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,1,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,1,0


### 领劵日期 Date_received

In [10]:
date_received = dfoff['Date_received'].unique()
date_received = sorted(date_received[date_received != 'null'])

date_buy = dfoff['Date'].unique()
date_buy = sorted(date_buy[date_buy != 'null'])

print('优惠卷收到日期从',date_received[0],'到',date_received[-1])
print('消费日期从',date_buy[0],'到',date_buy[-1])

优惠卷收到日期从 20160101 到 20160615
消费日期从 20160101 到 20160630


**关于领劵日期的特征：**

- weekday : {null, 1, 2, 3, 4, 5, 6, 7}

- weekday_type : {1, 0}（周六和周日为1，其他为0）

- Weekday_1 : {1, 0, 0, 0, 0, 0, 0}

- Weekday_2 : {0, 1, 0, 0, 0, 0, 0}

- Weekday_3 : {0, 0, 1, 0, 0, 0, 0}

- Weekday_4 : {0, 0, 0, 1, 0, 0, 0}

- Weekday_5 : {0, 0, 0, 0, 1, 0, 0}

- Weekday_6 : {0, 0, 0, 0, 0, 1, 0}

- Weekday_7 : {0, 0, 0, 0, 0, 0, 1}

In [12]:
def getWeekday(row):
    if row == 'null':
        return row
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)

# weekday_type :  周六和周日为1，其他为0
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x: 1 if x in [6,7] else 0)
dftest['weekday_type'] = dftest['weekday'].apply(lambda x: 1 if x in [6,7] else 0)

# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
#print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace('null', np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace('null', np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

In [13]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,distance,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,...,0,null,0,0,0,0,0,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,1,6,1,0,0,0,0,0,1,0
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,...,0,3,0,0,0,1,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,...,0,6,1,0,0,0,0,0,1,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,0,1,0,1,0,0,0,0,0,0


### 所有特征：

- discount_rate

- discount_type

- discount_man

- discount_jian

- distance

- weekday

- weekday_type

- weekday_1

- weekday_2

- weekday_3

- weekday_4

- weekday_5

- weekday_6

- weekday_7

## 标签标注

三种情况：

- Date_received == 'null'：表示没有领到优惠券，无需考虑，y = -1

- (Date_received != 'null') & (Date != 'null') & (Date - Date_received <= 15)：表示领取优惠券且在15天内使用，即正样本，y = 1

- (Date_received != 'null') & ((Date == 'null') | (Date - Date_received > 15))：表示领取优惠券未在在15天内使用，即负样本，y = 0

定义标签备注函数：

In [14]:
def label(row):
    if row['Date_received'] == 'null':
        return -1
    if row['Date'] != 'null':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff['label'] = dfoff.apply(label, axis=1)

In [15]:
print(dfoff['label'].value_counts())

 0    988887
-1    701602
 1     64395
Name: label, dtype: int64


In [16]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,...,null,0,0,0,0,0,0,0,0,-1
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,6,1,0,0,0,0,0,1,0,0
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,...,3,0,0,0,1,0,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,...,6,1,0,0,0,0,0,1,0,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,1,0,1,0,0,0,0,0,0,0


## 建立线性模型 SGDClassifier

- 使用上面提取的14个特征。

- 训练集：20160101-20160515；验证集：20160516-20160615。

- 用线性模型 SGDClassifier

### 划分训练集/验证集

In [17]:
# data split
df = dfoff[dfoff['label'] != -1].copy()
train = df[(df['Date_received'] < '20160516')].copy()
valid = df[(df['Date_received'] >= '20160516') & (df['Date_received'] <= '20160615')].copy()
print('Train Set: \n', train['label'].value_counts())
print('Valid Set: \n', valid['label'].value_counts())

Train Set: 
 0    759172
1     41524
Name: label, dtype: int64
Valid Set: 
 0    229715
1     22871
Name: label, dtype: int64


### 特征数量

In [18]:
# feature
original_feature = ['discount_rate','discount_type','discount_man', 'discount_jian','distance', 'weekday', 'weekday_type'] + weekdaycols
print('共有特征：',len(original_feature),'个')
print(original_feature)

共有特征： 14 个
['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


### 建立模型

In [19]:
def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log',  # loss function: logistic regression
        penalty='elasticnet', # L1 & L2
        fit_intercept=True,  # 是否存在截距，默认存在
        max_iter=100, 
        shuffle=True,  # Whether or not the training data should be shuffled after each epoch
        n_jobs=1, # The number of processors to use
        class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.
 
    # 管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。
    model = Pipeline(steps=[
        ('ss', StandardScaler()), # transformer
        ('en', classifier())  # estimator
    ])
 
    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }
 
    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1,  # -1 means using all processors
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

### 训练

In [20]:
predictors = original_feature
model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  2.3min finished


### 验证

对验证集中每个优惠券预测的结果计算 AUC，再对所有优惠券的 AUC 求平均。计算 AUC 的时候，如果 label 只有一类，就直接跳过，因为 AUC 无法计算。

In [21]:
# valid predict
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]
valid1.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label,pred_prob
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,1,0,0,0,0,0,1,0,0,0.019586
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,0,1,0,0,0,0,0,0,0,0.100966
6,1439408,2632,8591,20:1,0,20160516,20160613,1,0.950000,20,...,0,1,0,0,0,0,0,0,0,0.100966
9,2029232,450,1532,30:5,0,20160530,null,1,0.833333,30,...,0,1,0,0,0,0,0,0,0,0.096634
10,2029232,6459,12737,20:1,0,20160519,null,1,0.950000,20,...,0,0,0,0,1,0,0,0,0,0.132336


计算 AUC：

In [22]:
# avgAUC calculation
vg = valid1.groupby(['Coupon_id'])
aucs = []
for i in vg:
    tmpdf = i[1] 
    if len(tmpdf['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.532344469452


### 测试

In [23]:
# test prediction for submission
y_test_pred = model.predict_proba(dftest[predictors])
dftest1 = dftest[['User_id','Coupon_id','Date_received']].copy()
dftest1['Probability'] = y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head(5)

,User_id,Coupon_id,Date_received,Probability
0,4129537,9983,20160712,0.104786
1,6949378,3429,20160706,0.153189
2,2166529,6928,20160727,0.005538
3,2166529,1808,20160727,0.018725
4,6172162,6500,20160708,0.063459


### 保存模型 & 导入模型

In [ ]:
if not os.path.isfile('1_model.pkl'):
    with open('1_model.pkl', 'wb') as f:
        pickle.dump(model, f)
else:
    with open('1_model.pkl', 'rb') as f:
        model = pickle.load(f)

## 优化模型...

- **特征工程**

- **机器学习算法**

- **模型集成**

## 参考代码

[比赛第一名代码与解析](https://github.com/wepe/O2O-Coupon-Usage-Forecast)